<a href="https://colab.research.google.com/github/Ayesha2943/A-Basic-To-Do-Webapp/blob/main/my_mini_project_smart_vehicle_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving video.mp4 to video.mp4


In [ ]:
!pip install ultralytics
!pip install deep_sort_realtime
!pip install opencv-python-headless

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 107.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import math

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
cap = cv2.VideoCapture("video.mp4")

In [ ]:
from IPython.display import FileLink, display

display(FileLink("/content/output_result.mp4"))

/content/output_result.mp4

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import math
from google.colab.patches import cv2_imshow
from google.colab import files

# Load the YOLO model
model = YOLO("yolov8n.pt")

# Initialize DeepSORT
tracker = DeepSort(max_age=30)

# Function to check line intersection
def ccw(A, B, C):
    return (C[1] - A[1]) * (B[0] - A[0]) > (B[1] - A[1]) * (C[0] - A[0])

def intersect(A, B, C, D):
    return ccw(A, C, D) != ccw(B, C, D) and ccw(A, B, C) != ccw(A, B, D)

# Estimate speed
def estimate_speed(p1, p2, fps, ppm):
    distance_pixels = math.sqrt((p2[0] - p1[0])**2 + (p2[1] - p1[1])**2)
    distance_meters = distance_pixels / ppm
    speed_mps = distance_meters * fps
    speed_kmph = speed_mps * 3.6
    return speed_kmph

# Map YOLO class IDs to vehicle types
vehicle_classes = {
    2: "car",
    3: "motorcycle",
    5: "bus",
    7: "truck",
}

# Line coordinates
line_start = (100, 200)
line_end = (500, 200)

# Trackers and counters
object_prev_positions = {}
object_directions = {}
object_speeds = {}
vehicle_counts = {"car": 0, "motorcycle": 0, "bus": 0, "truck": 0}
counted_ids = set()

# Read input video
cap = cv2.VideoCapture("video.mp4")
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(3))
height = int(cap.get(4))
ppm = 10  # Adjust as needed

# Save the output
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter("output_result.mp4", fourcc, fps, (width, height))

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame, verbose=False)[0]
    detections = []

    for result in results.boxes.data.tolist():
        x1, y1, x2, y2, score, class_id = result
        class_id = int(class_id)
        if score > 0.5 and class_id in vehicle_classes:
            detections.append(([x1, y1, x2 - x1, y2 - y1], score, class_id))

    tracks = tracker.update_tracks(detections, frame=frame)

    for track in tracks:
        if not track.is_confirmed():
            continue

        track_id = track.track_id
        ltrb = track.to_ltrb()
        x1, y1, x2, y2 = map(int, ltrb)
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2

        # Estimate speed
        if track_id in object_prev_positions:
            prev_cx, prev_cy = object_prev_positions[track_id]
            speed = estimate_speed((prev_cx, prev_cy), (cx, cy), fps, ppm)
            object_speeds[track_id] = speed
        object_prev_positions[track_id] = (cx, cy)

        # Direction
        dx = cx - object_prev_positions[track_id][0]
        dy = cy - object_prev_positions[track_id][1]
        direction = "down" if dy > 0 else "up"
        object_directions[track_id] = direction

        # Count only once per ID
        if track_id not in counted_ids:
            prev_pos = object_prev_positions[track_id]
            if intersect(prev_pos, (cx, cy), line_start, line_end):
                class_id = track.det_class  # Get class from DeepSort detection
                vehicle_type = vehicle_classes.get(class_id, "unknown")
                if vehicle_type in vehicle_counts:
                    vehicle_counts[vehicle_type] += 1
                counted_ids.add(track_id)

        # Draw rectangle, label, speed
        class_id = track.det_class
        label = vehicle_classes.get(class_id, "unknown")
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"{label} ID:{track_id}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)
        if track_id in object_speeds:
            cv2.putText(frame, f"{object_speeds[track_id]:.2f} km/h", (x1, y2 + 15),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)

    # Draw line and display counts
    cv2.line(frame, line_start, line_end, (0, 0, 255), 2)
    y_offset = 30
    for v_type, count in vehicle_counts.items():
        cv2.putText(frame, f"{v_type.capitalize()}: {count}", (10, y_offset),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        y_offset += 25

    out.write(frame)

cap.release()
out.release()
print("✅ Video processing complete.")

# Download link
files.download("/content/output_result.mp4")


100%|██████████| 6.25M/6.25M [00:00<00:00, 351MB/s]


✅ Video processing complete.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files

# Download the video to your local computer
files.download("/content/output_result.mp4")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>